<a href="https://colab.research.google.com/github/Bcsaltdad/LoftCycleClubMatch/blob/main/LOFT_MATCHING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
import sqlite3  # You can replace with your preferred database connector
import random

app = Flask(__name__)
CORS(app)  # Enable CORS for cross-domain requests from your Squarespace site

# Database connection function - replace with your own database connection method
def get_db_connection():
    """Connect to your existing database"""
    # Example using SQLite - replace with your actual database connection
    conn = sqlite3.connect('your_database.db')
    conn.row_factory = sqlite3.Row
    return conn

@app.route('/recommend', methods=['POST'])
def recommend_classes():
    """Generate personalized class recommendations based on client preferences"""
    try:
        # Get client survey data from request
        data = request.json

        # Extract client preferences
        fitness_goals = data.get('fitness_goals', [])
        availability = data.get('availability', [])
        experience_level = data.get('experience_level', 'beginner')
        preferred_types = data.get('preferred_class_types', [])

        # Get classes from your database
        conn = get_db_connection()
        cursor = conn.cursor()

        # Step 1: Filter classes by availability (must match)
        availability_placeholders = ', '.join(['?'] * len(availability))
        availability_query = f"""
            SELECT * FROM classes
            WHERE time_slot IN ({availability_placeholders})
        """
        cursor.execute(availability_query, availability)
        available_classes = cursor.fetchall()

        # Convert to list of dictionaries for easier handling
        available_classes = [dict(cls) for cls in available_classes]

        # No classes match availability
        if not available_classes:
            return jsonify({
                'success': False,
                'message': 'No classes match your availability.',
                'recommendations': []
            })

        # Step 2: Score classes based on fitness goals
        scored_classes = []

        for cls in available_classes:
            # Start with base score
            score = 10

            # Increase score for matching fitness goals
            class_benefits = cls.get('benefits', [])
            if isinstance(class_benefits, str):
                # If benefits are stored as comma-separated string
                class_benefits = [b.strip() for b in class_benefits.split(',')]

            for goal in fitness_goals:
                if goal in class_benefits:
                    score += 5

            # Adjust score based on experience level and class intensity
            intensity_map = {'low': 1, 'medium': 2, 'high': 3}
            experience_map = {'beginner': 1, 'intermediate': 2, 'advanced': 3}

            intensity = intensity_map.get(cls.get('intensity', 'medium'), 2)
            exp_level = experience_map.get(experience_level, 1)

            # Beginners prefer lower intensity, advanced prefer higher
            intensity_match = 3 - abs(intensity - exp_level)
            score += intensity_match * 2

            # Boost score for preferred class types
            if cls.get('type') in preferred_types:
                score += 3

            # Add to scored list
            scored_classes.append({
                'class': cls,
                'score': score
            })

        # Step 3: Sort by score (highest first)
        scored_classes.sort(key=lambda x: x['score'], reverse=True)

        # Step 4: Ensure variety in recommendations (different class types)
        final_recommendations = []
        selected_types = set()

        # First, add highest scoring classes of different types
        for scored_class in scored_classes:
            cls = scored_class['class']
            cls_type = cls.get('type')

            if cls_type not in selected_types and len(final_recommendations) < 3:
                final_recommendations.append(cls)
                selected_types.add(cls_type)

        # If we need more classes to reach minimum recommendation count
        remaining_slots = 3 - len(final_recommendations)
        if remaining_slots > 0:
            # Get highest scoring remaining classes
            for scored_class in scored_classes:
                cls = scored_class['class']
                if cls not in final_recommendations and len(final_recommendations) < 5:
                    final_recommendations.append(cls)

                    # Stop at 5 classes maximum
                    if len(final_recommendations) >= 5:
                        break

        # Close the database connection
        conn.close()

        return jsonify({
            'success': True,
            'recommendations': final_recommendations
        })

    except Exception as e:
        return jsonify({
            'success': False,
            'message': f"Error generating recommendations: {str(e)}",
            'recommendations': []
        })

# Endpoint to test connection
@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({'status': 'ok'})

if __name__ == '__main__':
    # Get port from environment variable for cloud hosting compatibility
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port)